In [ ]:
POST /v2/vision/text/ocr HTTP/1.1
Host: dapi.kakao.com
Authorization: KakaoAK ${4622d29dc4040eec934a35986ddc9847}
Content-Type: multipart/form-data

In [3]:
import json
import cv2
import requests
import sys

LIMIT_PX = 1024
LIMIT_BYTE = 1024*1024  # 1MB
LIMIT_BOX = 40

rest_api_key = '4622d29dc4040eec934a35986ddc9847'

def kakao_ocr_resize(image_path: str):
    """
    ocr detect/recognize api helper
    ocr api의 제약사항이 넘어서는 이미지는 요청 이전에 전처리가 필요.

    pixel 제약사항 초과: resize
    용량 제약사항 초과  : 다른 포맷으로 압축, 이미지 분할 등의 처리 필요. (예제에서 제공하지 않음)

    :param image_path: 이미지파일 경로
    :return:
    """
    image = cv2.imread('image001.jpg')
    height, width, _ = image.shape

    if LIMIT_PX < height or LIMIT_PX < width:
        ratio = float(LIMIT_PX) / max(height, width)
        image = cv2.resize(image, None, fx=ratio, fy=ratio)
        height, width, _ = height, width, _ = image.shape

        # api 사용전에 이미지가 resize된 경우, recognize시 resize된 결과를 사용해야함.
        image_path = "{}_resized.jpg".format('image001.jpg')
        cv2.imwrite('image001.jpg', image)

        return 'image001.jpg'
    return None


def kakao_ocr(image_path: str, appkey: str):
    """
    OCR api request example
    :param image_path: 이미지파일 경로
    :param appkey: 카카오 앱 REST API 키
    """
    API_URL = 'https://dapi.kakao.com/v2/vision/text/ocr'

    headers = {'Authorization': 'KakaoAK {}'.format('4622d29dc4040eec934a35986ddc9847')}

    image = cv2.imread('image001.jpg')
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()


    return requests.post(API_URL, headers=headers, files={"image": data})


def main():
    if len(sys.argv) != 3:
        print("Please run with args: $ python example.py /path/to/image appkey")
    image_path, appkey = 'image001', '4622d29dc4040eec934a35986ddc9847'

    resize_impath = kakao_ocr_resize('image001.jpg')
    if resize_impath is not None:
        image_path = resize_impath
        print("원본 대신 리사이즈된 이미지를 사용합니다.")

    output = kakao_ocr('image001.jpg', '4622d29dc4040eec934a35986ddc9847').json()
    print("[OCR] output:\n{}\n".format(json.dumps(output, sort_keys=True, ensure_ascii=False, indent=2)))


if __name__ == "__main__":
    main()

[OCR] output:
{
  "result": [
    {
      "boxes": [
        [
          186,
          145
        ],
        [
          511,
          217
        ],
        [
          495,
          288
        ],
        [
          171,
          217
        ]
      ],
      "recognition_words": [
        "123가ø4568"
      ]
    },
    {
      "boxes": [
        [
          148,
          191
        ],
        [
          180,
          199
        ],
        [
          176,
          216
        ],
        [
          143,
          209
        ]
      ],
      "recognition_words": [
        "KOR"
      ]
    },
    {
      "boxes": [
        [
          553,
          431
        ],
        [
          681,
          431
        ],
        [
          681,
          452
        ],
        [
          553,
          452
        ]
      ],
      "recognition_words": [
        "CARISYOU"
      ]
    }
  ]
}

